In [6]:
!pip install pandas>None
!pip install rdkit>None
!pip install pubchempy>None
!pip install 'mordred[full]'

In [7]:
import pandas as pd
import pubchempy as pcp
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors

ModuleNotFoundError: No module named 'mordred'

In [ ]:
df = pd.read_csv('/home/student/Desktop/enam/data_preprocessing/1_algs.csv')

In [ ]:
df
df.info()

In [ ]:
# Получение списка доступных дескрипторов
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
num_descriptors = len(descriptor_names)

# Инициализация пустой матрицы для дескрипторов
descriptors_set = np.empty((0, num_descriptors), float)

# Расчет дескрипторов
for _, row in df.iterrows():
    smiles = row['smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
        descriptor = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
        descriptors_set = np.append(descriptors_set, descriptor, axis=0)

# Создание DataFrame с дескрипторами
df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)

# Объединение существующего датасета с новыми дескрипторами
df_rdkit = pd.concat([df, df_descriptors], axis=1)


In [6]:
df_rdkit
df_rdkit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 61 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       20000 non-null  int64  
 1   mol_id                           20000 non-null  object 
 2   smiles                           20000 non-null  object 
 3   A                                20000 non-null  float64
 4   B                                20000 non-null  float64
 5   C                                20000 non-null  float64
 6   mu                               20000 non-null  float64
 7   alpha                            20000 non-null  float64
 8   homo                             20000 non-null  float64
 9   lumo                             20000 non-null  float64
 10  gap                              20000 non-null  float64
 11  r2                               20000 non-null  float64
 12  zpve              

In [7]:
df_rdkit = df_rdkit.dropna(how='any')
print(df_rdkit)
print(df_rdkit.info())

       Unnamed: 0      mol_id                    smiles        A        B  \
0            3249    gdb_3250             C1CC11OC2CC12  6.20100  2.37357   
1           54069   gdb_54070  NC(=[NH2+])C([N-]C=O)C#N  2.34937  1.13704   
2           70760   gdb_70761             OC12CC1OCCOC2  2.49407  1.66830   
3           51650   gdb_51651            O=CNC1CC2CC2C1  3.56977  1.08344   
4          115918  gdb_115919             CCC1COC2CC2C1  3.51047  1.06282   
...           ...         ...                       ...      ...      ...   
19995       38825   gdb_38826          C1C2OC3CC=CCC123  2.95820  1.74625   
19996       39840   gdb_39841       C1C2C3OC3C3(CC3)N12  2.90099  1.80710   
19997       36633   gdb_36634           C1COC(=N1)NCC#N  6.14092  0.76405   
19998       68055   gdb_68056        CC12CC(C)(O1)C2C#C  2.05970  1.80984   
19999       32429   gdb_32430          Cc1c([nH]cc1NC)C  3.06619  1.18229   

             C      mu  alpha    homo    lumo  ...     chi0n     chi1n  \
0

In [8]:
print(type(df_rdkit['smiles']))

<class 'pandas.core.series.Series'>


In [5]:
calc = Calculator(descriptors, ignore_3D=True)
mols = []
for i in df_rdkit['smiles']:
    mol = Chem.MolFromSmiles(i)
    if mol is not None:
        mols.append(mol)

data_m = calc.pandas(mols)

df_all = pd.concat([df_rdkit, data_m], axis=1)

NameError: name 'Calculator' is not defined

In [ ]:
df_all.to_csv('/home/student/Desktop/enam/data_preprocessing/2_all_descriptors.csv')